In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import timing_analysis.lite_utils as lu
from timing_analysis.utils import check_recentness_noise
from timing_analysis.timingconfiguration import TimingConfiguration
from timing_analysis.plot_utils import plot_dmxout, plot_dmx_diffs_nbwb
import timing_analysis.report
from astropy import log
import glob
import os
from astropy.visualization import quantity_support
quantity_support()

# notebook gives interactive plots but not until the kernel is done
#%matplotlib notebook
# inline gives non-interactive plots right away
#%matplotlib inline

In [ ]:
# read in config files 
source = "[psr_name]"

nb_config = f"configs/{source}.nb.yaml"
wb_config = f"configs/{source}.wb.yaml"
nb_tc, wb_tc = TimingConfiguration(nb_config), TimingConfiguration(wb_config)
report = timing_analysis.report.Report(
    title=f"{source} NB-WB comparison", 
    sections=["Summary","DMX","Comparison","Warnings"])
report.begin_capturing_log("Warnings")
report.add_markdown("Summary", nb_tc.get_summary())
report.add_markdown("Summary", wb_tc.get_summary())

In [ ]:
# get nb model, toas, construct fitter
nb_mo,nb_to = nb_tc.get_model_and_toas(excised=True)
if not nb_to: nb_mo,nb_to = nb_tc.get_model_and_toas(apply_initial_cuts=True)
nb_tc.manual_cuts(nb_to)
nb_fo = nb_tc.construct_fitter(nb_to,nb_mo)

In [ ]:
# get wb model, toas, construct fitter
wb_mo,wb_to = wb_tc.get_model_and_toas(excised=True)
if not wb_to: wb_mo,wb_to = wb_tc.get_model_and_toas(apply_initial_cuts=True)
wb_tc.manual_cuts(wb_to)
wb_fo = wb_tc.construct_fitter(wb_to,wb_mo)

In [ ]:
plot_12p5_dmx = False

# get latest noise directories, dmxparse.?b.out files
nb_noise, nb_noise_list = check_recentness_noise(nb_tc)
wb_noise, wb_noise_list = check_recentness_noise(wb_tc)
nb_latest_noise, wb_latest_noise = nb_noise_list[-1], wb_noise_list[-1]

intermediate_dir = "/nanograv/share/15yr/timing/intermediate"
nb_dmxout = os.path.join(intermediate_dir,nb_latest_noise,"temp_files",f"{source}_dmxparse.nb.out")
wb_dmxout = os.path.join(intermediate_dir,wb_latest_noise,"temp_files",f"{source}_dmxparse.wb.out")

# look for 12.5 dmxout results, if desired
if plot_12p5_dmx:
    dmxout_12p5_dir = "/nanograv/share/15yr/timing/intermediate/dmxout_12p5/"
    dmxout_12p5 = glob.glob(os.path.join(dmxout_12p5_dir,f"{source}*.dmx"))
else:
    dmxout_12p5 = None

In [ ]:
# produce nb/wb dmx timeseries comparison (include excess dm due to solar wind model)
if dmxout_12p5:
    dmxDict = plot_dmxout([nb_dmxout,wb_dmxout,dmxout_12p5[0]],['nb','wb','12p5'], 
                          psrname=source, outfile = f"{source}_nbwb12p5_dmx.png",
                          model=nb_mo)
else:
    dmxDict = plot_dmxout([nb_dmxout,wb_dmxout],['nb','wb'], psrname=source, 
                          outfile = f"{source}_nbwb_dmx.png", model=nb_mo)
report.new_section("DMX")
report.add_plot("DMX", caption="DMX comparison")

In [ ]:
# inspect DMX outliers (identify problematic MJDs interactively)
dmx_outlier_mjds = [] # put your problem MJDs here!
match_files = lu.dmx_mjds_to_files(dmx_outlier_mjds,nb_to,dmxDict)

# all_GTpd=True by default to plot phase vs. frequency, but the kwarg/dropdown choices can be modified
if match_files:
    fsel_list, fpav_list, _tsel, _tpav = lu.display_excise_dropdowns(match_files, [], all_YFp=False, all_GTpd=True)

In [ ]:
# generate desired plots based on drop-down choices
if match_files:
    file_plot_list = lu.read_excise_dropdowns(fsel_list, fpav_list, match_files)
    lu.make_detective_plots(file_plot_list, match_files)

In [ ]:
# plot diffs if coincident nb and wb DMX measurements are available.
if ('nb' in dmxDict.keys()) and ('wb' in dmxDict.keys()):
    plot_dmx_diffs_nbwb(dmxDict, psrname=source, outfile=f"{source}_nbwb_dmxdiffs.png")
    report.add_plot("DMX", caption="DMX nb-wb diffs")
else:
    log.warning("Both nb and wb keys are not present in dmxDict; diffs not plotted.")

In [ ]:
# compare nb/wb models
compare_out = lu.compare_models(nb_fo,
                   model_to_compare=wb_tc.get_model_path(),
                   verbosity='max',
                   nodmx=True,
                   threshold_sigma=3.)
report.new_section("Comparison")
report.add_verbatim("Comparison", "\n".join(compare_out))

In [ ]:
report.generate_pdf(f"{source}_nbwb.pdf")
report.generate_html(f"{source}_nbwb.html")